In [50]:
# Imports
from credentials import password, email
from data import stock_list_one
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import datetime, re, requests, io, time, random, string

In [54]:
driver = webdriver.Chrome('/Users/Simone/Coding/chromedriver')
time.sleep(3)

In [55]:
def sign_in(email=email, password=password):
   driver.get('https://wallmine.com')
   time.sleep(2)
   driver.find_element_by_xpath('/html/body/main/header/div/ul/li[1]/ul/li[3]/a').click() # click sign in
   time.sleep(1)
   driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[1]/div[2]/a').click() #sign in with password
   # log in
   # verify on sign in page: 
   if "We're glad you're back!" in driver.page_source: 
       driver.find_element_by_xpath('//*[@id="user_email"]').send_keys(email)
       driver.find_element_by_xpath('//*[@id="user_password"]').send_keys(password)
       time.sleep(0.2)
       driver.find_element_by_xpath('//*[@id="new_user"]/div[5]/div[2]/div[1]/button').click()
       time.sleep(3)
       if 'US Markets' in driver.page_source: print('successfully signed in')
   else: 
        print('wrong page, start over')
        sign_in()

In [56]:
sign_in(email, password)

successfully signed in


In [64]:
broken_links = []
for stock in stock_list_one:
    driver.get(f"https://wallmine.com/{stock.get('exchange')}/{stock.get('stock_ticker')}")
    if "Sorry, we couldn't find the page you requested" in driver.page_source: broken_links.append(stock)
    time.sleep(1)
print(broken_links)

[{'stock_ticker': 'BRK', 'exchange': 'NYSE'}, {'stock_ticker': 'BRK', 'exchange': 'NYSE'}]


In [75]:
criteria = {
    "company_name" : '/html/body/main/section/div[2]/div/div[1]/h1/div[2]/a',
    'current_price' : '/html/body/main/section/div[3]/div/div/div/div/div[2]/div/div[1]/span[1]',
    'percentage' : '/html/body/main/section/div[3]/div/div/div/div/div[2]/div/div[2]/div',
    'amount_changed' : '/html/body/main/section/div[3]/div/div/div/div/div[2]/div/div[1]/span[2]',
    'market_cap' : '/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[1]/table/tbody/tr[1]/td/span',
    'enterprise_value' : '/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[1]/table/tbody/tr[2]/td/span',
    'income' : '/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[2]/table/tbody/tr[3]/td/span', 
    'revenue' : '/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[2]/table/tbody/tr[1]/td/span', 
    'ebitda' : '/html/body/main/section/div[4]/div[1]/div[2]/div[1]/div[2]/table/tbody/tr[2]/td/span',
    'volume' : '/html/body/main/section/div[4]/div[1]/div[2]/div[2]/div[1]/table/tbody/tr[1]/td',
    'relative_volume' : '/html/body/main/section/div[4]/div[1]/div[2]/div[2]/div[1]/table/tbody/tr[2]/td/span'
    }
company_data = {}
for attribute in criteria:
    company_data[attribute] = driver.find_element_by_xpath(criteria[attribute]).text
    # get percentage up or down
    if attribute == 'percentage': 
        if driver.find_element_by_xpath(criteria['percentage']).get_attribute('class') == 'badge badge-danger': company_data['percentage'] = '-' + company_data['percentage']
        else: company_data['percentage'] = '+' + company_data['percentage']
    if attribute in ['market_cap', 'income', 'revenue', 'ebitda', 'enterprise_value']:
        if company_data[attribute][-1] == 'B': 
            company_data[attribute] = float(company_data[attribute][1:-1]) * 1000000000
        elif company_data[attribute][-1] == 'M': 
            company_data[attribute] = float(company_data[attribute][1:-1]) * 1000000


print(company_data)


{'company_name': 'Qualcomm, Inc.', 'current_price': '$163.77', 'percentage': '+4.25%', 'amount_changed': '+6.68', 'market_cap': 172300000000.0, 'enterprise_value': 170540000000.0, 'income': 5198000000.0, 'revenue': 23532000000.0, 'ebitda': 7599000000.0, 'volume': '0.000 / 5.717M', 'relative_volume': '0.00 ×'}


In [ ]:
def get_data(stock_array):
    